#### CAFA6-11-Model Improvement

Improvement from CAFA6-08: use model ensemble (LogisticRegression + MLP)

---

In [1]:
!pip install biopython > /dev/null

The system cannot find the path specified.


## Step 1: Load CAFA6 files

---

In [4]:
# CAFA6 file paths
TRAIN_TERMS = "data/Train/train_terms.tsv"
TRAIN_SEQ = "data/Train/train_sequences.fasta"
TEST_SEQ = "data/Test/testsuperset.fasta"

In [5]:
from Bio import SeqIO

# Dict {entryId, seq}
train_sequences = {rec.id: str(rec.seq) for rec in SeqIO.parse(TRAIN_SEQ, 'fasta')}
test_sequences  = {rec.id: str(rec.seq) for rec in SeqIO.parse(TEST_SEQ,  'fasta')}

print(f'Loaded {len(train_sequences)} train and {len(test_sequences)} test sequences')

Loaded 82404 train and 224309 test sequences


In [6]:
print("Train dict:", list(train_sequences.items())[0])
print("Test dict:", list(test_sequences.items())[0])

Train dict: ('sp|A0A0C5B5G6|MOTSC_HUMAN', 'MRWQEMGYIFYPRKLR')
Test dict: ('A0A0C5B5G6', 'MRWQEMGYIFYPRKLR')


In [7]:
train_ids = [i.split('|')[1] for i in train_sequences.keys()]
test_ids = list(test_sequences.keys())

## Step 2: Feature extraction
  
---

In [8]:
# Embeddings file paths
ESM_EMBEDDINGS = "ESM2"
TRAIN_EMBEDDINGS = ESM_EMBEDDINGS + "/protein_embeddings_train.npy"
TEST_EMBEDDINGS = ESM_EMBEDDINGS + "/protein_embeddings_test.npy"

In [9]:
# Load embeddings
import numpy as np

X_train = np.load(TRAIN_EMBEDDINGS)
X_test = np.load(TEST_EMBEDDINGS)

In [10]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (82404, 320)
X_test shape: (224309, 320)


In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100, random_state=42)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced  = pca.transform(X_test)

In [12]:
print("X_train_reduced shape:", X_train_reduced.shape)
print("X_test_reduced shape:", X_test_reduced.shape)

X_train_reduced shape: (82404, 100)
X_test_reduced shape: (224309, 100)


In [13]:
# Normalization
from sklearn.preprocessing import StandardScaler

X_train_dict = {
    'MLP': {},
    'Logistic': {}
}

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
print("X_train_norm shape (MLP):", X_train_norm.shape)
print("X_test_norm shape (MLP):", X_test_norm.shape)

X_train_dict['MLP']['X_train'] = X_train_norm
X_train_dict['MLP']['X_test'] = X_test_norm

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train_reduced)
X_test_norm = scaler.transform(X_test_reduced)
print("X_train_norm shape (Logistic):", X_train_norm.shape)
print("X_test_norm shape (Logistic):", X_test_norm.shape)

X_train_dict['Logistic']['X_train'] = X_train_norm
X_train_dict['Logistic']['X_test'] = X_test_norm

X_train_norm shape (MLP): (82404, 320)
X_test_norm shape (MLP): (224309, 320)
X_train_norm shape (Logistic): (82404, 100)
X_test_norm shape (Logistic): (224309, 100)


## Step 3: Customized MLP

---

In [14]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [15]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.3),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.25),

            nn.Linear(1024, output_dim)  # logits
        )

    def forward(self, x):
        return self.net(x)

## Step 4: Label encoding + MLP training

---

In [16]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm

mlp_y_trains = dict()
mlp_mlb_dict = dict()
mlp_models = dict()

train_terms_df = pd.read_csv(TRAIN_TERMS, sep="\t")

for aspect in ['P', 'C', 'F']:
    # Filter the train_terms_df based on aspect
    ont_terms_df = train_terms_df[train_terms_df['aspect'] == aspect]

    # Group the dataFrame based on the EntryID, turn all the GO terms to a list, finally turns this dataFrame to a dict {entryID: terms}
    protein_terms = ont_terms_df.groupby('EntryID')['term'].apply(list).to_dict()

    # Create a list of labels for this aspect, if an entryID doesn't exist in this aspect, give it a []
    # This ensures y_train is of shape (82404, ...)
    labels = [protein_terms.get(entry_id, []) for entry_id in train_ids]

    # Multi-hot encoding, use sparse representation
    mlb = MultiLabelBinarizer(sparse_output=True)
    y_train = mlb.fit_transform(labels)
    mlp_y_trains[aspect] = y_train
    
    print(f"y_train shape for {aspect} ontology: {y_train.shape} \t\t Number of unique {aspect} terms: {y_train.shape[1]}")

    # Save to dict
    mlp_mlb_dict[aspect] = mlb
    model = MLPClassifier(input_dim=X_train.shape[1], output_dim=y_train.shape[1])
    mlp_models[aspect] = model

y_train shape for P ontology: (82404, 16858) 		 Number of unique P terms: 16858
y_train shape for C ontology: (82404, 2651) 		 Number of unique C terms: 2651
y_train shape for F ontology: (82404, 6616) 		 Number of unique F terms: 6616


In [17]:
mlp_models

{'P': MLPClassifier(
   (net): Sequential(
     (0): Linear(in_features=320, out_features=2048, bias=True)
     (1): ReLU()
     (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (3): Dropout(p=0.3, inplace=False)
     (4): Linear(in_features=2048, out_features=1024, bias=True)
     (5): ReLU()
     (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (7): Dropout(p=0.25, inplace=False)
     (8): Linear(in_features=1024, out_features=16858, bias=True)
   )
 ),
 'C': MLPClassifier(
   (net): Sequential(
     (0): Linear(in_features=320, out_features=2048, bias=True)
     (1): ReLU()
     (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (3): Dropout(p=0.3, inplace=False)
     (4): Linear(in_features=2048, out_features=1024, bias=True)
     (5): ReLU()
     (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (7): Dropout(p=0.25

In [18]:
mlp_y_trains

{'P': <Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 250805 stored elements and shape (82404, 16858)>,
 'C': <Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 157770 stored elements and shape (82404, 2651)>,
 'F': <Compressed Sparse Row sparse matrix of dtype 'int64'
 	with 128452 stored elements and shape (82404, 6616)>}

In [19]:
# DataLoader
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train_dict['MLP']['X_train'], dtype=torch.float32)

loaders = {}

for aspect in ['P', 'C', 'F']:
    # Convert sparse CSR → dense numpy → float tensor
    y_dense = mlp_y_trains[aspect].toarray().astype('float32')
    y_tensor = torch.tensor(y_dense, dtype=torch.float32)

    dataset = TensorDataset(X_train_tensor, y_tensor)

    loaders[aspect] = DataLoader(dataset, batch_size=128, shuffle=True)

    print(aspect, "loader ready:", y_tensor.shape)

P loader ready: torch.Size([82404, 16858])
C loader ready: torch.Size([82404, 2651])
F loader ready: torch.Size([82404, 6616])


In [20]:
# Training
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()

def train_one_model(model, loader, epochs=5, lr=1e-3, device='cuda'):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for ep in range(epochs):
        total_loss = 0.0

        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            logits = model(X_batch)               # shape: (batch, num_labels)
            loss = criterion(logits, y_batch)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {ep+1}/{epochs}   Loss = {total_loss/len(loader):.4f}")

    return model

In [23]:
# Train 3 models
trained_mlp_models = dict()

for aspect in ['P', 'C', 'F']:
    print("\nTraining", aspect, "...")
    trained_mlp_models[aspect] = train_one_model(
        model=mlp_models[aspect],
        loader=loaders[aspect],
        epochs=8,
        lr=1e-3,
        device=device
    )


Training P ...
Epoch 1/8   Loss = 0.0645
Epoch 2/8   Loss = 0.0017
Epoch 3/8   Loss = 0.0015
Epoch 4/8   Loss = 0.0015
Epoch 5/8   Loss = 0.0015
Epoch 6/8   Loss = 0.0014
Epoch 7/8   Loss = 0.0014
Epoch 8/8   Loss = 0.0013

Training C ...
Epoch 1/8   Loss = 0.0680
Epoch 2/8   Loss = 0.0038
Epoch 3/8   Loss = 0.0035
Epoch 4/8   Loss = 0.0034
Epoch 5/8   Loss = 0.0033
Epoch 6/8   Loss = 0.0032
Epoch 7/8   Loss = 0.0031
Epoch 8/8   Loss = 0.0031

Training F ...
Epoch 1/8   Loss = 0.0659
Epoch 2/8   Loss = 0.0016
Epoch 3/8   Loss = 0.0014
Epoch 4/8   Loss = 0.0013
Epoch 5/8   Loss = 0.0012
Epoch 6/8   Loss = 0.0012
Epoch 7/8   Loss = 0.0011
Epoch 8/8   Loss = 0.0011


## Step 5: Label encoding + Logistic Regression model training

---

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

logistic_mlb_dict = dict()
logistic_models = dict()

for aspect in tqdm(['P', 'C', 'F'], desc="Training Logistic Regression"):
    # Filter the train_terms_df based on aspect
    ont_terms_df = train_terms_df[train_terms_df['aspect'] == aspect]

    # Group the dataFrame based on the EntryID, turn all the GO terms to a list, finally turns this dataFrame to a dict {entryID: terms}
    protein_terms = ont_terms_df.groupby('EntryID')['term'].apply(list).to_dict()

    # Create a list of labels for this aspect, if an entryID doesn't exist in this aspect, give it a []
    # This ensures y_train is of shape (82404, ...)
    labels = [protein_terms.get(entry_id, []) for entry_id in train_ids]

    # Multi-hot encoding, use sparse representation
    mlb = MultiLabelBinarizer(sparse_output=True)
    y_train = mlb.fit_transform(labels)
    
    print(f"y_train shape for {aspect} ontology: {y_train.shape} \t\t Number of unique {aspect} terms: {y_train.shape[1]}")

    # Save to dict
    logistic_mlb_dict[aspect] = mlb
    model = OneVsRestClassifier(LogisticRegression(max_iter=600, solver='lbfgs', C=0.5, random_state=42), n_jobs=-1)
    model.fit(X_train_dict['Logistic']['X_train'], y_train)
    logistic_models[aspect] = model
    print(f"Model for {aspect} trained successfully.")

Training Logistic Regression:   0%|          | 0/3 [00:00<?, ?it/s]

y_train shape for P ontology: (82404, 16858) 		 Number of unique P terms: 16858


Training Logistic Regression:  33%|███▎      | 1/3 [38:01<1:16:02, 2281.48s/it]

Model for P trained successfully.
y_train shape for C ontology: (82404, 2651) 		 Number of unique C terms: 2651


Training Logistic Regression:  67%|██████▋   | 2/3 [44:01<19:11, 1151.40s/it]  

Model for C trained successfully.
y_train shape for F ontology: (82404, 6616) 		 Number of unique F terms: 6616


Training Logistic Regression: 100%|██████████| 3/3 [58:57<00:00, 1179.02s/it]

Model for F trained successfully.


## Step 6: Inference and Submission

In [25]:
BATCH_SIZE = 5000  # avoid memory overflow
mlp_submission_list = []

for i in tqdm(range(0, len(test_ids), BATCH_SIZE), desc="Predicting with MLP"):
    batch_entry_ids = test_ids[i : i + BATCH_SIZE]

    # Slice features
    X_batch = X_train_dict['MLP']['X_test'][i : i + BATCH_SIZE]
    X_batch = torch.tensor(X_batch, dtype=torch.float32, device=device)

    # For each ontology aspect (P, F, C)
    for aspect, model in trained_mlp_models.items():
        model.eval()

        # Forward pass (logits → probabilities)
        with torch.no_grad():
            logits = model(X_batch)
            probs = torch.sigmoid(logits).cpu().numpy()  # ndarray (batch, num_labels)

        mlb = mlp_mlb_dict[aspect]  # MultiLabelBinarizer for this aspect

        # Loop over proteins in the batch
        for j, entry_id in enumerate(batch_entry_ids):
            prob_vec = probs[j]

            # threshold at 0.02
            candidate_indices = np.where(prob_vec > 0.02)[0]

            for idx in candidate_indices:
                mlp_submission_list.append(
                    (entry_id, mlb.classes_[idx], round(prob_vec[idx], 3))
                )

Predicting with MLP: 100%|██████████| 45/45 [01:27<00:00,  1.94s/it]


In [26]:
print(f"{len(mlp_submission_list):,}")

5,664,932


In [27]:
BATCH_SIZE = 5000  # avoid memory overflow
logistic_submission_list = []

for i in tqdm(range(0, len(test_ids), BATCH_SIZE), desc="Predicting with Logistic Regression"):
    batch_entry_ids = test_ids[i : i + BATCH_SIZE]

    # Slice PCA-reduced features
    X_batch = X_train_dict['Logistic']['X_test'][i : i + BATCH_SIZE]

    for aspect, model in logistic_models.items():
        mlb = logistic_mlb_dict[aspect]
        y_pred_proba = model.predict_proba(X_batch)

        for j, entry_id in enumerate(batch_entry_ids):
            probs = y_pred_proba[j]
            candidate_indices = np.where(probs > 0.02)[0]

            for idx in candidate_indices:
                logistic_submission_list.append((entry_id, mlb.classes_[idx], round(probs[idx], 3)))

Predicting with Logistic Regression: 100%|██████████| 45/45 [42:02<00:00, 56.05s/it]


In [28]:
print(f"{len(logistic_submission_list):,}")

8,084,372


In [29]:
from collections import defaultdict

# Weighted ensemble. MLP: w, Logistic: (1-w).
# Weight computed from accuracy: 0.262 / (0.262 + 0.230) = 0.53
w = 0.53

merged_dict = defaultdict(lambda: [None, None])  
# [mlp_score, logistic_score]

# Add MLP predictions
for entry_id, go_term, score in tqdm(mlp_submission_list, desc="Merging MLP"):
    merged_dict[(entry_id, go_term)][0] = score

# Add Logistic predictions
for entry_id, go_term, score in tqdm(logistic_submission_list, desc="Merging Logistic"):
    merged_dict[(entry_id, go_term)][1] = score

# Final weighted merge
submission_list = []

for (entry_id, go_term), (mlp_score, log_score) in tqdm(merged_dict.items(), desc="Weighted averaging"):
    # treat missing values as 0 if they don't appear in one of the lists
    mlp_score = mlp_score if mlp_score is not None else 0.0
    log_score = log_score if log_score is not None else 0.0

    final_score = w * mlp_score + (1 - w) * log_score
    submission_list.append((entry_id, go_term, round(final_score, 3)))


Weighted averaging: 100%|██████████| 10196964/10196964 [00:34<00:00, 299548.67it/s]


In [30]:
submission_df = pd.DataFrame(
    submission_list,
    columns=['Protein Id', 'GO Term Id', 'Prediction']
)

submission_df = submission_df.sort_values(
    by=['Protein Id', 'Prediction'],
    ascending=[True, False]
)

# Limit 1500 predictions per protein
final_submission_df = (
    submission_df.groupby('Protein Id')
    .head(1500)
    .reset_index(drop=True)
)

final_submission_df.to_csv('submission.tsv', sep='\t', index=False, header=False)

In [31]:
print("\nSubmission file 'submission.tsv' created successfully.")
print(f"Total predictions in final submission: {len(final_submission_df):,}")
print("Submission DataFrame Head:")
display(final_submission_df.head())


Submission file 'submission.tsv' created successfully.
Total predictions in final submission: 10,196,836
Submission DataFrame Head:


,Protein Id,GO Term Id,Prediction
0,A0A017SE81,GO:0005515,0.160
1,A0A017SE81,GO:0008209,0.149
2,A0A017SE81,GO:0047044,0.143
3,A0A017SE81,GO:0005886,0.126
4,A0A017SE81,GO:0004303,0.111
